## 1. Setup & Configuration

In [1]:


import yaml
import os
import sys

# Windows compatibility
if sys.platform == "win32":
    import socketserver
    if not hasattr(socketserver, 'UnixStreamServer'):
        socketserver.UnixStreamServer = socketserver.TCPServer

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, count, countDistinct, desc, lit, avg, row_number
from pyspark.sql.functions import radians, sin, cos, sqrt, atan2, max as spark_max
from pyspark.sql.window import Window
import tempfile

# Load configuration
with open("../../config/config.yml") as f:
with open("../../config/config.yml") as f:
    config = yaml.safe_load(f)

buffer_m = config["params"]["buffer_m"]
bike_mode = config["filters"]["bike_mode_value"]
silver_dir = config["paths"]["silver_dir"]
gold_dir = config["paths"]["gold_dir"]

print(f"✓ Buffer distance: {buffer_m}m")
print(f"✓ Bike mode filter: {bike_mode}")
print(f"✓ Silver data: {silver_dir}")
print(f"✓ Gold output: {gold_dir}")

✓ Buffer distance: 200m
✓ Bike mode filter: velo
✓ Silver data: data/silver
✓ Gold output: data/gold


In [2]:
# Initialize Spark session
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
local_temp = tempfile.gettempdir()

spark = SparkSession.builder \
    .appName("Module2_SpatialUsage_Parquet") \
    .master("local[*]") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "localhost") \
    .config("spark.ui.enabled", "false") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.local.dir", local_temp) \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print(f"✓ Spark version: {spark.version}")
print(f"✓ HADOOP_HOME: {os.environ.get('HADOOP_HOME', 'Not set')}")

✓ Spark version: 4.0.1
✓ HADOOP_HOME: Not set


## 2. Load Silver Data (Parquet)

In [3]:
# Load Silver Parquet files (individual files, not partitioned directories)
silver_path = f"../../{silver_dir}"

df_amenagements = spark.read.parquet(f"{silver_path}/silver_amenagements_with_centroids")
df_sites = spark.read.parquet(f"{silver_path}/silver_sites")
df_channels = spark.read.parquet(f"{silver_path}/silver_channels")
df_measures = spark.read.parquet(f"{silver_path}/silver_measures_union2")

print("✓ Loaded Silver Parquet files:")
print(f"  - silver_amenagements: {df_amenagements.count()} rows")
print(f"  - silver_sites: {df_sites.count()} rows")
print(f"  - silver_channels: {df_channels.count()} rows")
print(f"  - silver_measures: {df_measures.count()} rows")

✓ Loaded Silver Parquet files:
  - silver_amenagements: 4592 rows
  - silver_sites: 280 rows
  - silver_channels: 1302 rows
  - silver_measures: 485651 rows


In [4]:
# Check existing parquet files in silver directory
import glob

print("Checking silver parquet files:")
for table in ["silver_amenagements", "silver_sites", "silver_channels"]:
    parquet_path = f"{silver_path}/{table}"
    try:
        df_temp = spark.read.parquet(parquet_path)
        print(f"{table} : {df_temp.count()}")
    except Exception as e:
        print(f"{table} : ERROR - {str(e)}")

=== Amenagements ===
+--------------+-----------------------------------------------+----------------------+------+--------+------+---------------------------------------+-------------------------------+-----------------------------------------+----------------+-----------------------------+---------------+-------------------------+------------+------------------+--------------------------------------------------------------+-----------------+-------------------------+----------------------+--------------+----------+-----------------------------------------------------------------------+--------+------------------+------------------+
|amenagement_id|nom                                            |commune1              |insee1|commune2|insee2|reseau                                 |financementac                  |typeamenagement                          |typeamenagement2|positionnement               |senscirculation|environnement            |localisation|typologiepiste    |revetementpis

In [ ]:
# Check silver_measures - it's partitioned by date
measures_path = f"{silver_path}/silver_measures"
try:
    df_measures_test = spark.read.parquet(measures_path)
    print(f"silver_measures : {df_measures_test.count()}")
    print("✓ All silver parquet files are accessible")
except Exception as e:
    print(f"silver_measures : ERROR - {str(e)}")

In [ ]:
# Load Silver Parquet files (individual files, not partitioned directories)
df_amenagements = spark.read.parquet(f"{silver_path}/silver_amenagements")
df_sites = spark.read.parquet(f"{silver_path}/silver_sites")
df_channels = spark.read.parquet(f"{silver_path}/silver_channels")

# For measures, we need to check if it's partitioned or if there's a union file
try:
    df_measures = spark.read.parquet(f"{silver_path}/silver_measures_union")
    measures_source = "silver_measures_union"
except:
    # Fall back to the partitioned silver_measures
    df_measures = spark.read.parquet(f"{silver_path}/silver_measures")
    measures_source = "silver_measures"

print("✓ Loaded Silver Parquet files:")
print(f"  - silver_amenagements: {df_amenagements.count()} rows")
print(f"  - silver_sites: {df_sites.count()} rows")
print(f"  - silver_channels: {df_channels.count()} rows")
print(f"  - {measures_source}: {df_measures.count()} rows")

In [ ]:
# Preview data and verify centroid columns
print("=== Amenagements ===")
print("Schema:")
df_amenagements.printSchema()

# Check if centroid_lat and centroid_lon exist
columns = df_amenagements.columns
has_centroid_lat = 'centroid_lat' in columns
has_centroid_lon = 'centroid_lon' in columns

print(f"✓ Has centroid_lat: {has_centroid_lat}")
print(f"✓ Has centroid_lon: {has_centroid_lon}")

if has_centroid_lat and has_centroid_lon:
    print("Sample data:")
    df_amenagements.select("amenagement_id", "nom", "centroid_lat", "centroid_lon").show(5)
else:
    print("⚠️ Missing centroid coordinates - showing available columns:")
    df_amenagements.show(5, truncate=False)

print("\n=== Sites ===")
df_sites.show(5)

print("\n=== Channels ===")
df_channels.show(5)

In [1]:
print("=== Measures ===")
df_measures.printSchema()
df_measures.show()
df_measures.dtypes

=== Measures ===


NameError: name 'df_measures' is not defined

## 3. Spatial Join: Link Counters to Infrastructure

Formule Haversine avec fonctions SQL natives (évite les UDFs Python qui crashent sur Windows)

In [6]:
# Haversine distance using NATIVE Spark SQL functions
def haversine_spark(lat1_col, lon1_col, lat2_col, lon2_col):
    """
    Calculate distance in meters between two points using Spark SQL native functions.
    """
    R = 6371000  # Earth radius in meters
    
    lat1_rad = radians(lat1_col)
    lon1_rad = radians(lon1_col)
    lat2_rad = radians(lat2_col)
    lon2_rad = radians(lon2_col)
    
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = sin(dlat / 2) ** 2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    return lit(R) * c

print("✓ Haversine function using native Spark SQL")

print("✓ Haversine function using native Spark SQL")



✓ Haversine function using native Spark SQL


In [7]:
# Cross join amenagements and sites
df_cross = df_amenagements.select(
    col("amenagement_id"),
    col("centroid_lat").alias("amen_lat"),
    col("centroid_lon").alias("amen_lon")
).crossJoin(
    df_sites.select(
        col("site_id"),
        col("lat").alias("site_lat"),
        col("lon").alias("site_lon")
    )
)

# Calculate distance and filter within buffer
df_within_buffer = df_cross.withColumn(
    "distance_m",
    haversine_spark(
        col("amen_lat"), col("amen_lon"),
        col("site_lat"), col("site_lon")
    )
).filter(col("distance_m") <= buffer_m)

print(f"✓ Found {df_within_buffer.count()} amenagement-site pairs within {buffer_m}m")
df_within_buffer.show()

✓ Found 1668 amenagement-site pairs within 200m
+--------------+------------------+-----------------+---------+------------------+------------------+------------------+
|amenagement_id|          amen_lat|         amen_lon|  site_id|          site_lat|          site_lon|        distance_m|
+--------------+------------------+-----------------+---------+------------------+------------------+------------------+
|          3751| 45.77711567620204|4.807585071234797|100042249| 45.77847191024936| 4.808332590112999|161.56505850594044|
|          3751| 45.77711567620204|4.807585071234797|100049670|  45.7784791437538| 4.808297837334199| 161.3731374960342|
|           403| 45.77132962612444|4.841625460871888|100030262| 45.77241907407597| 4.840592137379542|145.25278562002015|
|           403| 45.77132962612444|4.841625460871888|100030263|  45.7730227674639|4.8411329679323645|192.10460445536407|
|           403| 45.77132962612444|4.841625460871888|300016819|45.772428592809575|4.8409581184387225| 132

In [8]:
# Join with channels and filter for bike mode only
# Note: use 'is_bike_channel' column instead of 'mode'
df_amen_channels = df_within_buffer.join(
    df_channels,
    on="site_id",
    how="inner"
).filter(col("is_bike_channel") == True)

print(f"✓ Found {df_amen_channels.count()} amenagement-channel links (bike channels only)")
df_amen_channels.show()

✓ Found 4665 amenagement-channel links (bike channels only)
+---------+--------------+------------------+-----------------+------------------+------------------+------------------+----------+-------------+---------+------------+-----------+-------------------------+-----------------------------+---------+-------------------+--------+---------------+
|  site_id|amenagement_id|          amen_lat|         amen_lon|          site_lat|          site_lon|        distance_m|channel_id|mobility_type|direction|counter_type|temporality|counter_transmission_type|publication_transmission_type|time_step|         started_at|ended_at|is_bike_channel|
+---------+--------------+------------------+-----------------+------------------+------------------+------------------+----------+-------------+---------+------------+-----------+-------------------------+-----------------------------+---------+-------------------+--------+---------------+
|100042249|          3751| 45.77711567620204|4.807585071234797| 

In [9]:
# Create gold_link_amenagement_channel
gold_link = df_amen_channels.select(
    "amenagement_id",
    "channel_id",
    "site_id",
    "distance_m"
).distinct()

gold_link.cache()

print(f"✓ Created gold_link_amenagement_channel ({gold_link.count()} rows)")
gold_link.show()

✓ Created gold_link_amenagement_channel (4665 rows)
+--------------+----------+---------+------------------+
|amenagement_id|channel_id|  site_id|        distance_m|
+--------------+----------+---------+------------------+
|          3751| 101042249|100042249|161.56505850594044|
|           403| 101030262|100030262|145.25278562002015|
|           403| 353399193|100030262|145.25278562002015|
|           403| 353262889|300016819| 132.7092623578981|
|           403| 353262885|300016819| 132.7092623578981|
|          7077| 353500777|300048109|119.23687727763722|
|          7077| 353500776|300048109|119.23687727763722|
|          3699| 101030268|100030268| 198.4059164009821|
|          3699| 104060040|100060040|188.04276765464036|
|          3699| 102060040|100060040|188.04276765464036|
|          2219| 101034819|100034819|105.77434093505502|
|          2219| 353399205|100034819|105.77434093505502|
|          2864| 353427411|300039589|168.18705716219966|
|          2864| 353368811|300031957

## 4. Aggregate Daily Flows per Infrastructure

In [ ]:
from pyspark.sql.window import Window

# ==================================================================================
# ÉTAPE 1: Créer gold_link avec 1 SEUL site par aménagement (le plus proche)
# ==================================================================================
# D'abord, garder le site le plus proche pour chaque aménagement
window_amen_site = Window.partitionBy("amenagement_id").orderBy("distance_m")

gold_link_1site = gold_link.withColumn(
    "site_rank",
    row_number().over(window_amen_site)
).filter(
    col("site_rank") == 1  # Garder seulement le site le plus proche par aménagement
).drop("site_rank")

print(f"✓ Déduplication amenagement→site: {gold_link.select('amenagement_id').distinct().count()} amenagements")
print(f"  - Avant: {gold_link.count()} liens (amenagement-channel-site)")
print(f"  - Après: {gold_link_1site.count()} liens (1 site/amenagement)")

# Puis, garder 1 seul channel par site
window_channel = Window.partitionBy("site_id").orderBy("channel_id")

gold_link_unique = gold_link_1site.withColumn(
    "channel_rank",
    row_number().over(window_channel)
).filter(
    col("channel_rank") == 1  # Garder seulement le premier channel par site
).drop("channel_rank")

print(f"✓ Déduplication site→channel: {gold_link_unique.count()} liens finaux")

# Vérification: 1 site par aménagement
sites_per_amen = gold_link_unique.groupBy("amenagement_id").agg(
    countDistinct("site_id").alias("n_sites")
)
assert sites_per_amen.filter(col("n_sites") > 1).count() == 0, "ERREUR: Plus d'1 site par aménagement!"
print("✓ VALIDATION: 1 site par aménagement")

# ==================================================================================
# ÉTAPE 2: Lien measures → NEAREST site (évite duplicates inter-sites)
# ==================================================================================
df_measure_site_dist = df_measures.alias("m").join(
    df_sites.alias("s"),
    how="cross"
).withColumn(
    "distance_m",
    haversine_spark(
        col("m.lat"), col("m.lon"),
        col("s.lat"), col("s.lon")
    )
).select(
    col("m.point_id"),
    col("m.flux"),
    col("m.date"),
    col("m.point_type"),
    col("s.site_id"),
    col("distance_m")
)

# Garder seulement le site le plus proche pour chaque mesure
window_spec = Window.partitionBy("point_id", "date").orderBy("distance_m")

df_measures_with_site = df_measure_site_dist.withColumn(
    "rank", 
    row_number().over(window_spec)
).filter(
    col("rank") == 1  # Site le plus proche uniquement
).drop("rank", "distance_m")

print(f"\n✓ Linked {df_measures_with_site.count()} measures to NEAREST sites")

# ==================================================================================
# ÉTAPE 3: Jointure measures avec gold_link_unique (1 site/amenagement, 1 channel/site)
# ==================================================================================
df_flows = gold_link_unique.join(
    df_measures_with_site,
    on="site_id",
    how="inner"
).select(
    "amenagement_id",
    "channel_id", 
    "site_id",
    "date",
    "flux",
    "point_id",
    "point_type"
)

print(f"✓ Joined {df_flows.count()} measure records to amenagements")

# Vérification: un même (amenagement_id, date) ne doit apparaître qu'UNE FOIS
duplicate_check = df_flows.groupBy("amenagement_id", "date").count().filter(col("count") > 1)
n_duplicates = duplicate_check.count()
if n_duplicates > 0:
    print(f" WARNING: {n_duplicates} duplicates (amenagement_id, date) trouvés")
    duplicate_check.show(5)
else:
    print("✓ VALIDATION: No duplicates - 1 flux par (amenagement_id, date)")

print("\n=== Sample flow data ===")
df_flows.select("amenagement_id", "channel_id", "site_id", "date", "flux").show(10)

# Stats de contrôle
max_flux = df_flows.agg({"flux": "max"}).collect()[0][0]
print(f"\n Contrôle qualité:")
print(f"   Max flux: {max_flux:,} vélos/jour")
if max_flux > 20000:
    print(f"   ⚠️ Flux max > 20k - vérifier si réaliste pour Lyon")
else:
    print(f"   ✅ Flux max cohérent avec standards Lyon")

✓ Déduplication amenagement→site: 551 amenagements
  - Avant: 4665 liens (amenagement-channel-site)
  - Après: 551 liens (1 site/amenagement)
✓ Déduplication site→channel: 161 liens finaux
✓ VALIDATION: 1 site par aménagement

✓ Linked 485651 measures to NEAREST sites
✓ Joined 388035 measure records to amenagements
⚠️ WARNING: 2606 duplicates (amenagement_id, date) trouvés
+--------------+----------+-----+
|amenagement_id|      date|count|
+--------------+----------+-----+
|           303|2016-08-27|    2|
|           303|2016-09-10|    2|
|           303|2016-09-20|    2|
|           303|2016-10-12|    2|
|           303|2017-03-29|    2|
+--------------+----------+-----+
only showing top 5 rows


=== Sample flow data ===
+--------------+----------+---------+----------+----+
|amenagement_id|channel_id|  site_id|      date|flux|
+--------------+----------+---------+----------+----+
|          4195| 101013842|100013842|2014-08-06| 763|
|          4195| 101013842|100013842|2014-08-09| 59

In [11]:
# Aggregate by amenagement_id and date
gold_flow_daily = df_flows.groupBy("amenagement_id", "date").agg(
    sum("flux").alias("flux_estime"),
    countDistinct("channel_id").alias("n_channels")
).orderBy("amenagement_id", "date")

gold_flow_daily.cache()

print(f"✓ Created gold_flow_amenagement_daily ({gold_flow_daily.count()} rows)")
gold_flow_daily.show(20)

✓ Created gold_flow_amenagement_daily (385427 rows)
+--------------+----------+-----------+----------+
|amenagement_id|      date|flux_estime|n_channels|
+--------------+----------+-----------+----------+
|           129|2014-01-01|        161|         1|
|           129|2014-01-02|        185|         1|
|           129|2014-01-03|         94|         1|
|           129|2014-01-04|        125|         1|
|           129|2014-01-05|        281|         1|
|           129|2014-01-06|        309|         1|
|           129|2014-01-07|        384|         1|
|           129|2014-01-08|        378|         1|
|           129|2014-01-09|        298|         1|
|           129|2014-01-10|        311|         1|
|           129|2014-01-11|        281|         1|
|           129|2014-01-12|        220|         1|
|           129|2014-01-13|        191|         1|
|           129|2014-01-14|        245|         1|
|           129|2014-01-15|        267|         1|
|           129|2014-01-16|   

## 5. Data Quality Checks

In [ ]:
# Check 1: No duplicate amenagement-channel links
duplicate_links = gold_link.groupBy("amenagement_id", "channel_id").count().filter(col("count") > 1)
assert duplicate_links.count() == 0, "FAILED: Found duplicate amenagement-channel links"
print("✓ No duplicate links")

# Check 2: Count amenagements with data
amen_with_data = gold_flow_daily.select("amenagement_id").distinct().count()
print(f"✓ {amen_with_data} amenagements with flow data")

# Check 3: flux_estime should be non-negative
negative_flux = gold_flow_daily.filter(col("flux_estime") < 0)
assert negative_flux.count() == 0, "FAILED: Found negative flux values"
print("✓ All flux values are non-negative")

# Check 4: n_channels should be >= 1
zero_channels = gold_flow_daily.filter(col("n_channels") < 1)
assert zero_channels.count() == 0, "FAILED: Found days with 0 channels"
print("✓ All days have at least 1 channel")

print("\n All quality checks passed!")

✓ No duplicate links
✓ 153 amenagements with flow data
✓ All flux values are non-negative
✓ All days have at least 1 channel

🎉 All quality checks passed!


## 6. Summary Statistics

In [13]:
# Summary by amenagement
summary = gold_flow_daily.groupBy("amenagement_id").agg(
    count("date").alias("total_days"),
    sum("flux_estime").alias("total_flux"),
    avg("flux_estime").alias("avg_daily_flux"),
    spark_max("n_channels").alias("max_channels")
).orderBy(desc("avg_daily_flux"))

print("Summary by infrastructure:")
summary.show()

Summary by infrastructure:
+--------------+----------+----------+------------------+------------+
|amenagement_id|total_days|total_flux|    avg_daily_flux|max_channels|
+--------------+----------+----------+------------------+------------+
|          4112|      4266|  28745766| 6738.341772151899|           1|
|          4265|      3567|  22088867| 6192.561536305018|           1|
|          2845|      4120|  22937475| 5567.348300970873|           1|
|          4513|       806|   3739698|  4639.82382133995|           1|
|          4093|      3275|  14053186| 4291.049160305343|           1|
|          2941|      3905|  15820933| 4051.455313700384|           1|
|          3527|      2291|   9255362|4039.8786556089044|           1|
|          3539|      3567|  13833507|3878.1909167367535|           1|
|          4295|       619|   2370289|3829.2229402261714|           1|
|          1909|      3556|  13077269|3677.5222159730033|           1|
|          4177|      4326|  15599451|3605.9757281

## 7. Save Outputs (Parquet)

In [14]:
# Create output directory
gold_path = f"../../{gold_dir}"
os.makedirs(gold_path, exist_ok=True)

# Save as Parquet
gold_link.write.mode("overwrite").parquet(f"{gold_path}/gold_link_amenagement_channel")
print(f"✓ Saved gold_link_amenagement_channel.parquet")

gold_flow_daily.write.mode("overwrite").parquet(f"{gold_path}/gold_flow_amenagement_daily")
print(f"✓ Saved gold_flow_amenagement_daily.parquet")

print(f"\n All Gold outputs saved as Parquet to {gold_path}/")

✓ Saved gold_link_amenagement_channel.parquet
✓ Saved gold_flow_amenagement_daily.parquet

 All Gold outputs saved as Parquet to ../../data/gold/


In [15]:
# Verify saved files
print("=== Vérification des fichiers Gold Parquet ===")
for table in ["gold_link_amenagement_channel", "gold_flow_amenagement_daily"]:
    df = spark.read.parquet(f"{gold_path}/{table}")
    print(f"{table}: {df.count()} rows, {len(df.columns)} columns")
    df.printSchema()

=== Vérification des fichiers Gold Parquet ===
gold_link_amenagement_channel: 4665 rows, 4 columns
root
 |-- amenagement_id: string (nullable = true)
 |-- channel_id: string (nullable = true)
 |-- site_id: string (nullable = true)
 |-- distance_m: double (nullable = true)

gold_flow_amenagement_daily: 385427 rows, 4 columns
root
 |-- amenagement_id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- flux_estime: long (nullable = true)
 |-- n_channels: long (nullable = true)



## 8. Cleanup

In [16]:
# Stop Spark
spark.stop()
print("✓ Spark session stopped")

✓ Spark session stopped


## 9. Structure des Fichiers

### Entrées (Silver Parquet)
```
data/silver/
├── silver_amenagements/
├── silver_sites/
├── silver_channels/
└── silver_measures/
```

### Sorties (Gold Parquet)
```
data/gold/
├── gold_link_amenagement_channel/
└── gold_flow_amenagement_daily/
```
